In [3]:
import numpy as np
import pandas as pd
import matplotlib as plt
import random

In [4]:
def simulate_round(budget: int, original_bet: int):
    bet = original_bet
    while budget >= bet:
        flip = random.randint(0,1)
        if flip == 0:   #Heads
            return budget + bet
        else:   #Tails
            budget -= bet
            bet *= 2
    return budget

In [30]:
def simulate_strategy(budget: int, original_bet: int, goal: int):
    history = [budget]

    while budget > 0 and budget < goal:
        budget = simulate_round(budget, original_bet)
        history.append(budget)
    return history

In [36]:
print(simulate_strategy(100, 1, 200))

[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200]


In [28]:
res = 0
for _ in range(1000000):
    res += simulate_round(10, 1) == 11
print(res)

874852
